In [93]:
import folium
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize

In [94]:
kol_lat = 22.5726
kol_lon = 88.3639
print('The coordinates of Kolkata, India is {} N, {} E'.format(kol_lat, kol_lon))

The coordinates of Kolkata, India is 22.5726 N, 88.3639 E


In [95]:
kol_map = folium.Map(location=[kol_lat, kol_lon], zoom_start=13)
bang_map = folium.Map(location=[bang_lat, bang_lon], zoom_start=12)

In [96]:
bang_lat = 12.9716
bang_lon = 77.5946
print('The coordinates of Bangaluru, India is {} N, {} E'.format(bang_lat, bang_lon))

The coordinates of Bangaluru, India is 12.9716 N, 77.5946 E


In [97]:
Creds = pd.read_csv('FourSquareCredentials.csv')
CLIENT_ID = Creds['CLIENT_ID'][0] # your Foursquare ID
CLIENT_SECRET = Creds['CLIENT_SECRET'][0] # your Foursquare Secret
VERSION = Creds['VERSION'][0] # Foursquare API version

In [113]:
radius = 6000
LIMIT = 200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    kol_lat, 
    kol_lon, 
    radius, 
    LIMIT)

In [114]:
results = requests.get(url).json()

{'meta': {'code': 200, 'requestId': '5d3e9f985fc8ed0031922fea'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kolkata',
  'headerFullLocation': 'Kolkata',
  'headerLocationGranularity': 'city',
  'totalResults': 99,
  'suggestedBounds': {'ne': {'lat': 22.626600054000054,
    'lng': 88.42227078882908},
   'sw': {'lat': 22.51859994599995, 'lng': 88.30552921117092}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d4c107ee1ec6dcbad95d475',
       'name': 'The Oberoi Grand',
       'location': {'address': '15, Jawaharlal Nehru Rd',
        'lat': 22.56174943131244,
        'lng': 88.35159438973265,
        'labeledLatLngs': [{'label': 'display',
          'lat': 22.56174943131244,


In [100]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [150]:
def get_venue_rating(VENUE_ID):
    url2 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
        VENUE_ID,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        kol_lat, 
        kol_lon, 
        radius, 
        LIMIT)
    results2 = requests.get(url2).json()
    return VENUE_ID, results2['response']['venue']['rating']

In [151]:
venues = results['response']['groups'][0]['items']
    
kol_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
kol_venues =kol_venues.loc[:, filtered_columns]

# filter the category for each row
kol_venues['venue.categories'] = kol_venues.apply(get_category_type, axis=1)
# clean columns
kol_venues.columns = [col.split(".")[-1] for col in kol_venues.columns]

kol_venues.shape

(99, 5)

In [152]:
for lat, lng, cat, name in zip(kol_venues['lat'], kol_venues['lng'], kol_venues['categories'], kol_venues['name']):
    label = '{} ({})'.format(name, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='##3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(kol_map)  
    
kol_map

In [153]:
kol_venues.sample(10)

,id,name,categories,lat,lng
26,4bb4f2d60fb7b7132cb9bf8a,ITC Sonar,Hotel,22.544284,88.398017
37,5ab0e6b16adbf502ef89feb3,Starbucks Coffee,Coffee Shop,22.552444,88.353199
2,4bd1a0bd5e0cce72d3bda184,Lalit Great Eastern Hotel,Hotel,22.567967,88.350010
85,4dcf98d6d22deadedd6fb095,Kaafila,Indian Restaurant,22.587875,88.408374
22,4d19f40dbb488cfa6628c1d4,Beijing Restaurant,Chinese Restaurant,22.546518,88.387213
0,4d4c107ee1ec6dcbad95d475,The Oberoi Grand,Hotel,22.561749,88.351594
58,4b81757cf964a52075a730e3,The Cream & Fudge Factory,Bakery,22.534549,88.364854
56,579747adcd1069eeb483cc39,The GRID,Brewery,22.541975,88.386362
46,4bc788f793bdeee1917d37ae,Hotel Hindusthan International,Hotel,22.541075,88.350490
3,4e2c0d7a18a80bb0585fae4d,Blue & Beyond,Pub,22.559131,88.353280


In [154]:
id_ = []
rating = []
for _,row in kol_venues.iterrows():
    id_.append(row['id'])
    rating.append(get_venue_rating(row['id']))
temp_df = {'id': id_, 'rating':rating}
temp_df = pd.Dataframe(temp_df)

KeyError: 'venue'

In [135]:
kol_venues.head()

,id,name,categories,lat,lng,rating
0,4d4c107ee1ec6dcbad95d475,The Oberoi Grand,Hotel,22.561749,88.351594,0
1,4bc9c62b511f9521982faec7,Eden Garden,Cricket Ground,22.564542,88.343296,0
2,4bd1a0bd5e0cce72d3bda184,Lalit Great Eastern Hotel,Hotel,22.567967,88.350010,0
3,4e2c0d7a18a80bb0585fae4d,Blue & Beyond,Pub,22.559131,88.353280,0
4,4c284edefe6e2d7f5417533c,Peter Cat,Indian Restaurant,22.552365,88.352544,0


In [144]:
results2['response']['venue']['rating']

8.9